In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import PolynomialFeatures

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn import tree
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.linear_model import Lasso

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

In [22]:
df = pd.read_csv('IphoneData.csv')
newdf = df.drop(columns=['NAME', 'CO2E'])
features = newdf.to_numpy()
emissions = df['CO2E'].to_numpy()

In [3]:
def modelScore(X, y, model):
    avgR2 = avgSSE = avgMAPE = 0
    #cross validation using K folds
    kFolds = 5
    for i, (trainIndex, testIndex) in enumerate(KFold(n_splits=kFolds, shuffle=True, random_state=42).split(X)):
        X_train = X[trainIndex]
        X_test = X[testIndex]
        y_train = y[trainIndex]
        y_test = y[testIndex]
    

        if model == 'linear regression':
            fittedModel = LinearRegression().fit(X_train, y_train)
            
        elif model == 'decision tree':
            fittedModel = tree.DecisionTreeRegressor().fit(X_train, y_train)
                
        elif model == 'random forest':
            y_train = y_train.flatten()
            RF = RandomForestRegressor(max_features='log2', n_estimators=200, max_depth=20)
            fittedModel = RF.fit(X_train, y_train)
            
        elif model == 'support vector regression':
            y_train = y_train.flatten()
            fittedModel = SVR(kernel = 'linear', C=1).fit(X_train, y_train)
    
        elif model == 'polynomial regression':
            polyX_train = PolynomialFeatures(degree=2).fit_transform(X_train)
            polyX_test = PolynomialFeatures(degree=2).fit_transform(X_test)
            fittedModel = LinearRegression().fit(polyX_train, y_train)
            X_test = polyX_test

        elif model == 'xgboost':
            xgbModel = xgb.XGBModel(learning_rate=0.225, n_estimators=225, booster='gblinear',
                                        reg_alpha=0, reg_lambda=0, eval_metric='auc', 
                                        objective='reg:squaredlogerror')
            fittedModel = xgbModel.fit(X_train, y_train)
        
        elif model =='lasso':
            fittedModel = Lasso(alpha=0.1, selection='random', tol=0.019, max_iter=2000).fit(X_train, y_train)
            

        y_pred = fittedModel.predict(X_test)

        #scores
        avgR2 += r2_score(y_test, y_pred)    
        avgSSE += np.sum((y_test - y_pred) ** 2)
        avgMAPE += mean_absolute_percentage_error(y_test, y_pred)

    avgR2, avgSSE, avgMAPE = avgR2 / kFolds, avgSSE / kFolds, avgMAPE / kFolds
   #print results
    print(model)
    print("R2 = " + str(avgR2))
    print("SSE = " + str(avgSSE))
    print("MAPE = " + str(avgMAPE))




In [23]:
modelScore(features, emissions, 'linear regression')

linear regression
R2 = 0.9132785915590604
SSE = 172.61956963452138
MAPE = 0.046142026222792024


In [24]:
modelScore(features, emissions, 'decision tree')

decision tree
R2 = 0.6988395748454794
SSE = 665.6
MAPE = 0.0846360491960162


In [25]:
modelScore(features, emissions, 'random forest')

random forest
R2 = 0.7326601365568322
SSE = 630.2738012499999
MAPE = 0.08467118382189727


In [26]:
modelScore(features, emissions, 'support vector regression')

KeyboardInterrupt: 

In [27]:
modelScore(features, emissions, 'polynomial regression')

polynomial regression
R2 = 0.588444928232592
SSE = 1147.2346015235657
MAPE = 0.0678812670319829


In [32]:
modelScore(features, emissions, 'xgboost')

xgboost
R2 = 0.9085195472446195
SSE = 197.16225199967158
MAPE = 0.04685144251959759


In [35]:
modelScore(features, emissions, 'lasso')

lasso
R2 = 0.9012450943328796
SSE = 202.49435634423517
MAPE = 0.049850387062321526


In [20]:
modelsList = ['linear regression', 'decision tree', 'random forest', 'support vector regression', 'polynomial regression', 'xgboost', 'lasso']
multipleFeatures = df[['DISPLAY (inch)', 'STORAGE (gb)', 'MEMORY (gb)', 'GPU (cores)', 'NEURAL ENGINE (cores)', 'WEIGHT (gm)', 'FRONT CAMERA (mp)', 'BATTERY (mAh)', 'number of camera (back)', 'Sum of megapixels', 'MAIN CAMERA (mp)']].to_numpy()
featuresList = ['DISPLAY (inch)', 'STORAGE (gb)', 'MEMORY (gb)', 'CPU (cores)', 'GPU (cores)', 'NEURAL ENGINE (cores)', 'WEIGHT (gm)', 'FRONT CAMERA (mp)', 'BATTERY (mAh)', 'number of camera (back)', 'Sum of megapixels', 'MAIN CAMERA (mp)']

In [21]:
for model in modelsList:
    modelScore(multipleFeatures, emissions, model)

linear regression
R2 = 0.9132785915590595
SSE = 172.6195696345176
MAPE = 0.04614202622279172
decision tree
R2 = 0.6702082134996548
SSE = 672.8
MAPE = 0.08589700081778787
random forest
R2 = 0.7310037404653574
SSE = 628.8759516666668
MAPE = 0.08424676295610725
support vector regression
R2 = 0.8783187181737919
SSE = 231.72822054432027
MAPE = 0.04743814956132754
polynomial regression
R2 = 0.588444928232335
SSE = 1147.2346015240382
MAPE = 0.06788126703185489
xgboost
R2 = 0.9070973232287682
SSE = 199.14509507845213
MAPE = 0.046950983376304316
lasso
R2 = 0.9012475590830398
SSE = 202.5540328558383
MAPE = 0.04992929738073974


In [ ]:
"""for x in featuresList:
    print(x)
    for model in modelsList:
        print(modelScore(df[x].to_numpy().reshape(-1, 1), emissions, model))
    print("")  """

'for x in featuresList:\n    print(x)\n    for model in modelsList:\n        print(modelScore(df[x].to_numpy().reshape(-1, 1), emissions, model))\n    print("")  '